# Data Cleanup

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_json('raw_data.json')
df.head()

,url,name,type,price,prefecture,smv,acidity,kw_word,kw_wines,kw_beer,kw_foods,description
0,https://www.truesake.com/collections/all/produ...,"Dassai 45 Junmai Daiginjo ""Otter Festival""",Junmai Daiginjo,30.00,Yamaguchi,+3,1.4,Chunky,Pinot Noir / Chewy Whites,Ales,"Mushroom risotto, tempura, fried chicken, cavi...",This sake has an older brother Daiginjo that h...
1,https://www.truesake.com/collections/all/produ...,"Kikusui Junmai Ginjo ""Chrysanthemum Water""",Junmai Ginjo,33.00,Niigata,+1,1.6,Citrus,Sharp Whites/Dry Reds,Ales/Ambers,Suitable for light and stronger flavored dishe...,True Sake owner Beau Timken recorded K...
2,https://www.truesake.com/collections/all/produ...,"Wakatake Onikoroshi Junmai Daiginjo ""Demon Sla...",Junmai Daiginjo,49.00,Shizuoka,+/-0,1.4,Popular,Elegant reds/Complex whites,Crisp Ales,Cuisines that like big white wines.,Talk about a nice aroma package on thi...
3,https://www.truesake.com/collections/all/produ...,"Shunnoten Tokubetsu Junmai ""Fishermen Cup""",Tokubetsu Junmai,9.00,Yamanashi,+3,1.6,Smooth,Pinot Noir/Slippery Whites,Gentle Ales,None,The nose on this awesome looking “catch cup” i...
4,https://www.truesake.com/collections/all/produ...,"Kenbishi Kuromatsu Honjozo ""Black Pine""",Honjozo,40.00,Hyogo,±0.5,1.5,Rich,Deep Reds/Fat Whites,Stouts,Perfect for pub food salty and savory and grea...,The nose on this extremely famous Honjozo from...


First, I'm curious to see how well the description parsing worked: 

In [3]:
df.kw_word.unique()

array(['Chunky ', 'Citrus ', 'Popular ', 'Smooth ', 'Rich ', 'Even ',
       None, 'Hops ', 'Consistent ', 'Big ', 'Pear ', 'Peach ', 'Velvet ',
       '23% ', 'Vivacious ', 'Full Monty ', 'Dry      ', 'Gentle ',
       'Light ', 'Cedar ', 'Strawberry ',
       "Superb WINE: Zesty Dry Reds/Crisp Whites BEER: Crisp Ales. FOODS: The kitchen sink! Super food friendly. Updated 2019/02\xa0Vivid, voluptuous and vibrant sake in a can? You bet! This exclusive unpasteurized brew is Beau Timken's prized sake offering for True Sake customers. With a nose of strawberry, watermelon, sweet rice and cinnamon this undiluted sake (18.5%) is fat, happy, bright and chewy. Fruity, but not overly sweet, think big thoughts when tasting this velvety Nama that is pure pleasure in a can. (Yes cans keep sake colder longer!) This abundantly flavored sake is available only at True Sake! WORD: Jolly ",
       'Clean ', 'Chewy', 'Big Clean ', 'Dry ', 'Milky ', 'Beefy ',
       'Amber ', 'Flavorful ', 'Floral\xa0', 

Looks like I have a few listings where it didn't work so well... let's take a look: 

In [4]:
df[df['kw_word'].str.len() > 40]

,url,name,type,price,prefecture,smv,acidity,kw_word,kw_wines,kw_beer,kw_foods,description
23,https://www.truesake.com/collections/all/produ...,"Narutotai Ginjo Nama Genshu ""Red Snapper""",Ginjo Nama Genshu,38.00,Tokushima,+5,1.7,Superb WINE: Zesty Dry Reds/Crisp Whites BEER:...,Zesty Dry Reds/Crisp Whites BEER: Crisp Ales. ...,Crisp Ales. FOODS: The kitchen sink! Super foo...,The kitchen sink! Super food friendly. Updated...,We used to refer to this wonderful Can as “Bea...
123,https://www.truesake.com/collections/all/produ...,"Masumi Arabashiri Junmai Ginjo Nama ""First Run""",Junmai Ginjo Nama,urrent price $25.00,Nagano,-0.6,1.7,Lively WINE: Tannin Reds/ Crisp Whites BEER: C...,Tannin Reds/ Crisp Whites BEER: Citrus Ales FO...,"Citrus Ales FOODS: International Spice, Champa...","International Spice, Champagne Fare, Grilled f...",The nose on this very famous seasonally releas...
165,https://www.truesake.com/collections/all/produ...,"Harushika Shiboribana Junmai Ginjo Nama ""Sprin...",Junmai Ginjo Nama,36.00,Nara,-2,1.8,Elegant WINE: Soft Reds/Clean Whites BEER: Lig...,Soft Reds/Clean Whites BEER: Light Beer FOODS:...,"Light Beer FOODS: Clean cuisines, grilled fare...","Clean cuisines, grilled fare, shellfish, sashi...",The nose on this just released unpasteurized s...
194,https://www.truesake.com/collections/all/produ...,"Masumi Junmai Ginjo Hiyaoroshi ""Sleeping Beauty""",Junmai Ginjo Hiyaoroshi,41.00,Nagano,±0,1.8,Balanced WINE: Rich Reds/Fat Whites BEER: Belg...,Rich Reds/Fat Whites BEER: Belgian Ales FOODS:...,Belgian Ales FOODS: Fall cooked meats and fowl...,"Fall cooked meats and fowl, grilled fish, smok...",The nose on this seasonally released “Draft Sa...
203,https://www.truesake.com/collections/all/produ...,Nechi Otokoyama Tokubetsu Honjozo Nama “Yukimi...,Tokubetsu Honjozo Nama,urrent price $32.00,Niigata,+4,1.8,Clean WINE: Soft Reds/ Dry Whites BEER: Light ...,Soft Reds/ Dry Whites BEER: Light Beer FOODS: ...,"Light Beer FOODS: Salty, oily, savory grilled ...","Salty, oily, savory grilled fare, fried foods,...",The nose on this seasonally released unpasteur...
214,https://www.truesake.com/collections/all/produ...,Shirataki Jozen Junmai Ginjo Hiyaoroshi,Junmai Ginjo Hiyaoroshi,42.00,Niigata,±0,1.5,Rich WINE: Deep Reds/Fat Whites BEER: Fruity A...,Deep Reds/Fat Whites BEER: Fruity Ales FOODS: ...,Fruity Ales FOODS: All the wonderful full-bodi...,All the wonderful full-bodied and complex autu...,The nose on this seasonally released single-pa...
219,https://www.truesake.com/collections/all/produ...,Shichida Hiyaoroshi Junmai,Hiyaoroshi Junmai,44.00,Saga,+1,1.6,Berry Pie WINE: Deep Reds/Fruity Whites BEER: ...,Deep Reds/Fruity Whites BEER: Fruity Ales FOOD...,Fruity Ales FOODS: Rich and full-bodied Autumn...,"Rich and full-bodied Autumn cuisines, fatty fi...",The nose on this Fall Draft seasonally release...


In [5]:
for link in df[df['kw_word'].str.len() > 40].url:
    print(link)

https://www.truesake.com/collections/all/products/narutotai-nama-genshu
https://www.truesake.com/collections/all/products/masumi-arabashiri
https://www.truesake.com/collections/all/products/harushika-shiboribana
https://www.truesake.com/collections/all/products/masumi-junmai-ginjo-hiyaoroshi-sleeping-beauty
https://www.truesake.com/collections/all/products/nechi-otokoyama-tokubetsu-honjozo-nama-yukimizake
https://www.truesake.com/collections/all/products/shirataki-jozen-junmai-ginjo-hiyaoroshi
https://www.truesake.com/collections/all/products/shichida-hiyaoroshi-omachi-junmai


After reviewing these links, these products have revised descriptions. Basically duplicating the description section, including the 4 keywords. 

I'll do some data cleanup for these 7 products.

In [6]:
def split_on_chars(chars, text):
    if text is not None:
        if chars in text:
            return text.split(chars)[0].strip()
        else:
            return text.strip()
    else:
        return text

In [7]:
df['kw_word'] = df['kw_word'].apply(lambda description: split_on_chars('WINE', description))

In [8]:
df.kw_word.unique()

array(['Chunky', 'Citrus', 'Popular', 'Smooth', 'Rich', 'Even', None,
       'Hops', 'Consistent', 'Big', 'Pear', 'Peach', 'Velvet', '23%',
       'Vivacious', 'Full Monty', 'Dry', 'Gentle', 'Light', 'Cedar',
       'Strawberry', 'Superb', 'Clean', 'Chewy', 'Big Clean', 'Milky',
       'Beefy', 'Amber', 'Flavorful', 'Floral', 'Grapes –', 'Fat',
       'Earthy', 'Divine', 'Velvety', 'Balanced', 'Desirable', 'Special',
       'Layered', 'Yuzu', 'Creamsicle', 'Juicy', 'Bright', 'Coconut.',
       'Waves', 'Lively', 'Cocoa', 'Soft', 'Elegant', 'Solid',
       'Structured', 'Juice Box', 'Complex', 'Clever', 'Melon', 'Umami',
       'Effervescent', 'Mushroom', 'Gooey', 'Historical', 'Snappy',
       'Slick', 'Fruity', 'Stellar', 'Ripe', 'Lush', 'Fruit Basket',
       'Liquid Umami', 'Expressive', 'Shiitake Mushroom', 'Crisp',
       'Subtle', 'Voluptuous', 'Professional', 'Slippery', 'Brisk',
       'Clean Wine: Cabs/crisp whites', 'Smooth Cedar', 'Chocolate',
       'Brown Sugar', 'Sunflowe

Next - `kw_wines` column:

In [9]:
df['kw_wines'] = df['kw_wines'].apply(lambda description: split_on_chars('BEER', description))

In [10]:
df.kw_wines.unique()

array(['Pinot Noir / Chewy Whites', 'Sharp Whites/Dry Reds',
       'Elegant reds/Complex whites', 'Pinot Noir/Slippery Whites',
       'Deep Reds/Fat\xa0Whites', 'Dry reds/gentle whites', None,
       'Dry Whites/Strong Reds', 'Dry Whites and Tannin-laced reds',
       'Huge reds and sweet whites.', 'White Burgundy/Sauvignon Blanc',
       'Merlots/Sauvignon Blanc', 'Plump Reds/Smooth Whites',
       'Merlot/soft Chardonnay', 'French Pinot Noir/French Chardonnay',
       'Zins and Cabs', 'Smooth Reds/ Round whites',
       'Pinot Noir/White Burgundy', 'easy reds/dry whites',
       'Soft reds/round whites', 'Very Cedary Whites',
       'Fruity reds/ Chardonnay', 'Zesty Dry Reds/Crisp Whites',
       'Sauvignon Blancs/Dry Reds', 'Huge Reds/Massive Whites',
       'Merlot/Pinot Noir/Sauvignon Blanc', 'Clean reds/Dry whites',
       'Merlot', 'Chardonay', 'Big reds/Huge whites',
       'Thick Reds/Chewy Whites', 'Massive Dry Reds/Crisp Tight Whites',
       'Soft Reds/Fat whites', 'Zesty

Onto `kw_beer` column:

In [11]:
df['kw_beer'] = df['kw_beer'].apply(lambda description: split_on_chars('FOOD', description))

In [12]:
df.kw_beer.unique()

array(['Ales', 'Ales/Ambers', 'Crisp Ales', 'Gentle Ales', 'Stouts',
       'light beers', None, 'Hoppy Ales/Pilsners', 'High alc ales',
       'Ambers/Smooth Stouts', 'Amber/Honey Ales', 'Soft Ales',
       'Soft ales', 'Deep Belgians', 'Belgian Ales', 'Clean Ales',
       'Creamy Ales', 'soft ales', 'Sweet ales', 'Special Seasonal Brews',
       'Fruity ales', 'Crisp Ales.', 'Blond Ales/Hefeweizen',
       'Light Ales.', 'Creamy ales', 'Strong ales', 'Ambers/Honey Stouts',
       'IPA', 'Pale Ales –', 'Sharp ales', 'Hefeweizen', 'Soft dry ales',
       'Ales/pilsners', 'Light Ales', 'Clean ales', 'Light beers',
       'Rich Ales', 'Citrus pale ales', 'Crisp ales', 'Honey Ales',
       'Pilsners', 'Crisp Fruity Ales', 'Belgians', 'Gentle ales',
       'Honey Wheat Beers', 'Creamy Belgians', 'Pilsners/Ambers',
       'Large ales', 'Balanced ales', 'fruity ales', 'Earthy Pilsners',
       'Sweet honey ales', 'Sweet Ales', 'big ales.', 'Light ales',
       'Fruity Ales', 'rich ales', 'Sm

In [13]:
df['kw_beer'] = df['kw_beer'].apply(lambda description: split_on_chars('BEER:', description))
df['kw_beer'] = df['kw_beer'].apply(lambda description: split_on_chars('Foods:', description))

In [14]:
df.kw_beer.unique()

array(['Ales', 'Ales/Ambers', 'Crisp Ales', 'Gentle Ales', 'Stouts',
       'light beers', None, 'Hoppy Ales/Pilsners', 'High alc ales',
       'Ambers/Smooth Stouts', 'Amber/Honey Ales', 'Soft Ales',
       'Soft ales', 'Deep Belgians', 'Belgian Ales', 'Clean Ales',
       'Creamy Ales', 'soft ales', 'Sweet ales', 'Special Seasonal Brews',
       'Fruity ales', 'Crisp Ales.', 'Blond Ales/Hefeweizen',
       'Light Ales.', 'Creamy ales', 'Strong ales', 'Ambers/Honey Stouts',
       'IPA', 'Pale Ales –', 'Sharp ales', 'Hefeweizen', 'Soft dry ales',
       'Ales/pilsners', 'Light Ales', 'Clean ales', 'Light beers',
       'Rich Ales', 'Citrus pale ales', 'Crisp ales', 'Honey Ales',
       'Pilsners', 'Crisp Fruity Ales', 'Belgians', 'Gentle ales',
       'Honey Wheat Beers', 'Creamy Belgians', 'Pilsners/Ambers',
       'Large ales', 'Balanced ales', 'fruity ales', 'Earthy Pilsners',
       'Sweet honey ales', 'Sweet Ales', 'big ales.', 'Light ales',
       'Fruity Ales', 'rich ales', 'Sm

Onto `kw_foods` column: 

In [15]:
df.kw_foods.unique()

array(['Mushroom risotto, tempura, fried chicken, caviar, smoked salmon pate.',
       'Suitable for light and stronger flavored dishes such as dim sum, chicken/pork with mild spice, creamy cheeses and fresh fruits.      ',
       'Cuisines that like big white wines.      ', None,
       'Perfect for pub food salty and savory and\xa0greasy, also soup and stews when warmed.',
       'Bar snacks and grilled fare.      ',
       'Vegetable tempura, albacore sashimi, salads with ginger-soy dressings.      ',
       'Miso-based dishes, Sashimi, Stir-fries.      ',
       'All spice dishes from anywhere in the world, salty fare, pub snacks.      ',
       'Shrimp tempura, scallops with broccoli, sake steamed clams, Japanese omelets.\xa0',
       'Steamed white fish, fried scampi, daikon stew.      ',
       'Clean fare, elegant flavors, fruit salads, avocado salads, cheese plates.',
       'Best to drink as is!      ',
       'Oily and grilled fare, sushi, shellfish.      ',
       'Forget a

In [16]:
df['kw_foods'] = df['kw_foods'].apply(lambda description: split_on_chars('Updated', description))
df['kw_foods'] = df['kw_foods'].apply(lambda description: split_on_chars('(', description))

In [17]:
df.prefecture.unique()

array(['Yamaguchi', 'Niigata', 'Shizuoka', 'Yamanashi', 'Hyogo', 'Osaka',
       'Kouchi', 'Hokkaido', 'Yamagata', 'Tottori', 'Tochigi', 'Fukui',
       'Akita', 'Tokushima', 'Ohita', 'Wakayama', 'Shimane', 'Ishikawa',
       'Fukushima', 'Aomori', 'Kyoto', 'Iwate', 'Saitama', 'Gifu',
       'Nagano', 'Oakland, California', 'Shiga', 'Ehime', 'Kochi',
       'Miyagi', 'Ibaraki', 'Aichi', 'Hiroshima', 'Saga', 'Nanago',
       'Fukuoka', 'Nara', 'Domestic - San Francisco', None, 'Okayama',
       'Okinawa', 'Tokyo', 'Shizouka', 'Gunma', 'Kanagawa'], dtype=object)